In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# Lê os arquivos csv

In [2]:
data_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
data_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
data_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

# Pré processando o texto

In [3]:

from sklearn.feature_extraction.text import TfidfVectorizer



#instância do TfidfVectorizer com pré-processamento personalizado
tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,               #coloca tudo em minúsculas
    stop_words='english',         #remove stopwords em inglês
    token_pattern=r'(?u)\b\w\w+\b',  #usa um padrão regex para tokens
    max_features=5000,             #limita o número máximo de recursos
    ngram_range = (1,2)
)

#ajustar e transformar os dados de treinamento usando o vetorizador TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['text'])

#transformar os dados de teste usando o mesmo vetorizador TF-IDF
X_test_tfidf = tfidf_vectorizer.transform(data_test['text'])

# Treinando o modelo de fato

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

#cria um modelo MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 64), activation='relu', max_iter=150, random_state=0)

#treina o modelo com os dados de treinamento do csv,utilizando o target como rótulo da resposta correta
mlp_model.fit(X_train_tfidf, data_train['target'])

#faz previsões com base nos dados de treinamento
y_pred = mlp_model.predict(X_train_tfidf)

#avalia o desempenho do modelo comparando as previsões com os rótulos dados
report = classification_report(data_train['target'], y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4342
           1       0.98      0.97      0.98      3271

    accuracy                           0.98      7613
   macro avg       0.98      0.98      0.98      7613
weighted avg       0.98      0.98      0.98      7613



# Explicando o report

**Precision:** avalia a precisão das previsões positivas do modelo. Uma alta precisão significa que o modelo raramente faz previsões falsas positivas.

**Recall**: avalia a capacidade do modelo de identificar todos os exemplos positivos. Um alto recall significa que o modelo raramente deixa de identificar exemplos positivos.

**F1-Score**: é uma métrica que combina precisão e recall em um único valor. É particularmente útil quando as classes não estão balanceadas, ou seja, quando uma classe tem muito mais exemplos do que a outra.


# Submetendo arquivo

In [5]:
#extraindo os IDs do arquivo de teste para usar no submission
ids = data_test['id']

#fazendo previsões nos dados de teste
y_pred = mlp_model.predict(X_test_tfidf)

#dataframe de submissão com os IDs e as previsões de teste
submission_data = pd.DataFrame({'id': ids, 'target': y_pred})

#salva o dataframe de submissão em um arquivo CSV chamado "sample_submission.csv"
submission_data.to_csv('/kaggle/working/sample_submission.csv', index=False)

